In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime

In [348]:
df = pd.read_csv("L:\\ML-Assignment\\training_data.csv")

In [349]:
splits = list(df.groupby("Sentiment Level"))


negative = splits[0][1]
positive = splits[1][1]

In [350]:
import random as rand

number = rand.randint(0,500000)
positive = positive[number : number + 50000]

number = rand.randint(0,500000)
negative = negative[number: number + 50000]

In [351]:
df_concat = pd.concat([positive, negative], axis = 0)

In [352]:
df_concat = df_concat.sample(frac = 1)

In [353]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_concat["joined_Tweet"], df_concat["Sentiment Level"], random_state = 0)

In [354]:
x_train = np.asarray(x_train.to_frame().to_numpy(dtype=np.string_)).astype(np.string_)
x_test = np.asarray(x_test.to_frame().to_numpy(dtype=np.string_)).astype(np.string_)
y_train = np.asarray(y_train.to_frame().to_numpy(dtype=np.string_).astype(np.string_))
y_test = np.asarray(y_test.to_frame().to_numpy(dtype=np.string_).astype(np.string_))


def create_label(y):
    counter = 0
    result = np.array([[]])
    if y[0] == np.string_("0"):
        result = np.array([[1,0]])
    elif y[0] == np.string_("4"):
        result = np.array([[0,1]])
    
    for i in y: 
        if counter > 0:
            if i == np.string_("0"):
                result = np.append(result, np.array([[1,0]]), axis = 0)
            elif i == np.string_("4"):
                result = np.append(result, np.array([[0,1]]), axis = 0)
        counter+=1
    

    return result


new_y_train = create_label(y_train)   
new_y_test = create_label(y_test)

                

In [4]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, load_model 
from tensorflow.keras.layers import Dense, TextVectorization
from sklearn.metrics import accuracy_score

In [197]:
MAX_LEN = 200
MAX_FEATURES = 200000 


In [368]:
model = Sequential(name = "Nice_model")
model.add(Input(shape=(1,), dtype = tf.string))
vectorize_layer = TextVectorization(split = "whitespace", 
                                    ngrams=(1,2),
                                    output_mode = "tf_idf")

vectorize_layer.adapt(np.asarray(df_concat["joined_Tweet"].to_numpy(dtype=np.string_)).astype(np.string_))

model.add(vectorize_layer)

model.add(Dense(units = 128, activation = "relu", name = "L1"))
model.add(Dense(units = 128, activation = "relu", name = "L2"))
model.add(Dense(units = 128, activation = "relu", name = "L3"))
model.add(Dense(units = 64, activation = "relu", name = "L4"))
model.add(Dense(units = 2, activation = "sigmoid", name = "Output"))

model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics = "accuracy")

print("OK")

OK


In [369]:
model.summary()

Model: "Nice_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_17 (Text  (None, 484946)           1         
 Vectorization)                                                  
                                                                 
 L1 (Dense)                  (None, 128)               62073216  
                                                                 
 L2 (Dense)                  (None, 128)               16512     
                                                                 
 L3 (Dense)                  (None, 128)               16512     
                                                                 
 L4 (Dense)                  (None, 64)                8256      
                                                                 
 Output (Dense)              (None, 2)                 130       
                                                        

In [ ]:
score = model.evaluate(x_test, new_y_text)

print(score[0])
print(score[1])

In [373]:
log_dir = "L:\\ML-Assignment\\Logs\\Logs" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram freq = 1)

model.fit(x_train, new_y_train,
         epochs=5,
         batch_size=50,
         callbacks=[tensorboard_callback],
         verbose = 1,
         validation_data = (x_test, new_y_test))


Epoch 1/5
1500/1500 [==============================] - 912s 608ms/step - loss: 0.4643 - accuracy: 0.7892 - val_loss: 0.4478 - val_accuracy: 0.8005
Epoch 2/5
1500/1500 [==============================] - 817s 544ms/step - loss: 0.2894 - accuracy: 0.8879 - val_loss: 0.4888 - val_accuracy: 0.7947
Epoch 3/5
1500/1500 [==============================] - 827s 551ms/step - loss: 0.1394 - accuracy: 0.9544 - val_loss: 0.5938 - val_accuracy: 0.7895
Epoch 4/5
1500/1500 [==============================] - 831s 554ms/step - loss: 0.0707 - accuracy: 0.9794 - val_loss: 0.7775 - val_accuracy: 0.7835
Epoch 5/5
1500/1500 [==============================] - 811s 541ms/step - loss: 0.0495 - accuracy: 0.9860 - val_loss: 0.8972 - val_accuracy: 0.7750


In [374]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 25004), started 3:50:19 ago. (Use '!kill 25004' to kill it.)

In [1]:
model_dir = "L:\\ML-Assignment\\Model\\Model_02"
model.save(model_dir)

NameError: name 'model' is not defined

In [376]:
del model

In [5]:
model = load_model(model_dir)

print("Loaded")

Loaded


In [272]:
print(new_y_train[6])
print(x_train[6])

[1 0]
[b'awesome see speaking WordCampNOLA Wish I lived closer']


In [6]:
import contractions
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np 

stopwords = set(stopwords.words('english'))

def predict(model, input, stopword = stopwords):
    input = input.lower()
    input = re.sub("@[^\s]+", "", input)
    input = re.sub('r[^\w\s]+', "", input)
    input = input.split(" ")
    
    
    emoji = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
    "]+", re.UNICODE)
    
    input = [re.sub(emoji,"",x) for x in input]
    input = [x.encode("ascii","ignore").decode() for x in input]
    input = [contractions.fix(x) for x in input]
    input = ' '.join(map(str, input))
    input = word_tokenize(input, language = 'english')
    #input = [x for x in input if x not in stopwords]
    input = [x for x in input if not x.isdigit()]
    input = ' '.join(map(str, input))
    filtered_text = input
    
    x_input = np.array([input], dtype = np.string_)
    y = model.predict(x_input)
    result = ""
    
    if y[0][0] == y[0].max():
        result = "Depression Detected"
    elif y[0][1] == y[0].max():
        result = "No Depression"
        
    return (result, y, filtered_text)




In [ ]:
while True: 
    x = input("Enter Text: ")
    
    if x == "exit":
        break
    
    y = predict(model, x)
    print("Filtered Text: " + y[2])
    print("Model Result: " + y[0])
    print("Depression Detected: ", y[1][0][0])
    print("No Depression: ", y[1][0][1])
    
print("Completed.")

Enter Text: i had a biology class today and i cried the soul out of me hey. as the title says, today i had a biology class about miscarriages and stuff. now this wouldn't be so bad if it weren't for the fact that yesterday i was told that my dad's wife had not one but two miscarriages, one being before my brother was born (my brother is a rainbow baby) and the other one last friday. i felt like shit when i was told this, and this fucking class made it even worse. i excused myself, went to the bathroom and cried for like 20 minutes.  my little brother is only 4 and he doesn't like me. i was hoping they'd have another baby that would be nicer, but now it's dead. if its any use for anyone, both fetuses died at 2 months. edit: my brother is actually my step-brother  might not seem like a big deal to some people but here i am
1/1 [==============================] - 0s 146ms/step
Filtered Text: i had a biology class today and i cried the soul out of me hey . as the title says , today i had a 

1/1 [==============================] - 0s 35ms/step
Filtered Text: my boyfriends mom killed herself and she was an amazing person i loved her alot ( my own mom and i do not have a good relationship ) so it was nice to have a motherly figure who understood me . she killed herself and left several notes . she explained that she felt like nobody listened because she was dumb which she was not . she also said she felt like the odd one out since she can remembe she was on antidepressants and she showed no signs of doing something like this anytime soon . anyways i have struggled with suicidal thoughts and depression since i was so i related to her and what she wrote . only i related so much i feel like she was maybe the only person who described this feeling i have had for so long . this made me feel like there is no getting better for someone like me . i feel like this is exactly my faith she had kids one of them being yrs . even children did not keep her going . i feel like i am going to 